In [40]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import itertools
import random
from matplotlib import pyplot as plt
from pair_trading_foundations.data_generation import ExecutePairTrading, generate_training_data
random.seed(23)
import cProfile
import pstats
import pickle
import plotly.express as px
from time import time

def chunker(seq, size):
    # split a list into chunks
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [34]:
data = pd.read_csv('Data/sp500_full_20181231_to_20231229.csv')

In [35]:
value_count_tb = data[['Ticker']].groupby('Ticker').size().reset_index()
value_count_tb.columns = ['Ticker', 'Count']
stock_to_keep = value_count_tb['Ticker'][value_count_tb.Count==value_count_tb.Count.max()]
data = data[data.Ticker.isin(stock_to_keep)]
# sampled_tickers = random.sample(list(stock_to_keep.values), 10)
# data_tech = data[data['GICS Sector'].isin(['Information Technology'])]
# data_sampled = data[data['Ticker'].isin(sampled_tickers)]

In [36]:
tickers = list(set(data.Ticker.values))
combinations = list(itertools.combinations(tickers, 2))

In [37]:
batches = list(chunker(combinations, 5000))

In [38]:
print(f"Used around {15/45}s per pair for features of full history")
print(f"Used around {30/45}s per pair for features and labels of full history")

Used around 0.3333333333333333s per pair for features of full history
Used around 0.6666666666666666s per pair for features and labels of full history


# Generate for all pairs

In [ ]:
i = 0
for batch in batches:
    start_ts=time()
    print(f'Getting {i+1}th out of {len(batches)} batches')
    features_tb, labels_tb, pnl_metadata_tb = generate_training_data(
        data=data,
        training_len=500,
        test_len=120,
        calculate_label=True,
        verbose=False,
        combinations=batch
    )
    combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)
    combined = pd.merge(combined, pnl_metadata_tb[['Date', 'Ticker_P1','Ticker_P2', 'trade_executions']], how='left', on=['Date', 'Ticker_P1','Ticker_P2'])
    # combined = combined[combined.pnls.notnull()].reset_index(drop=True)
    combined.to_csv(f'Data/Training/pair_features{i+1}.csv', index=False)
    end_ts = time()
    print(f"Took {end_ts - start_ts} seconds")

Getting 1th out of 24 batches
5000 stock pairs detected
Took 0.1360929012298584 to initilize. Entering ticker pair loop


/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  features_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:330: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  labels_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations

# Generate data for pairs

In [7]:
features_tb, labels_tb, pnl_metadata_tb = generate_training_data(
        data=data_sampled,
        training_len=500,
        test_len=120,
        calculate_label=True,
        verbose=False
    )

45 stock pairs detected
Took 0.012228965759277344 to initilize. Entering ticker pair loop


/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:287: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  features_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations/notebooks/pair_trading_foundations/data_generation.py:329: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  labels_tb = pd.concat(
/Users/xiaoma/Desktop/current_desktop/mids/pair_trading/pair-trading-foundations

Took 29.999696969985962 to finish


In [8]:
combined = pd.merge(features_tb, labels_tb, how='left', on=['Date', 'Ticker_P1','Ticker_P2']).reset_index(drop=True)

In [9]:
combined = pd.merge(combined, pnl_metadata_tb[['Date', 'Ticker_P1','Ticker_P2', 'trade_executions']], how='left', on=['Date', 'Ticker_P1','Ticker_P2'])

In [10]:
with open('Data/spotcheckout_output.pkl','wb') as file:
    pickle.dump(combined, file)